In [1]:
from collections import defaultdict
import numpy as np
from util.plot import plot_cdf
from util.func import decode_dict
import matplotlib.pyplot as plt

In [2]:
dir_input="/data2/lichenni/path_tc"
dir_input_str=''
topo_type="_topo-pl-x_"
lr=10
enable_longest_foreground_traffic=True

data_list=[]
for shard in np.arange(10):
# for shard in [99]:
    for n_flows in [20000]:
        for n_hosts in [3,5,7]:
            for shard_seed in [0,1,2,3]:
                topo_type_cur=topo_type.replace("-x_",f"-{n_hosts}_")+"s%d"%(shard_seed)
                spec=f"shard{shard}_nflows{n_flows}_nhosts{n_hosts}_lr{lr}Gbps{dir_input_str}"
                dir_input_tmp = f"{dir_input}/{spec}"
                stats=decode_dict(np.load(f'{dir_input_tmp}/stats.npy', allow_pickle=True,encoding="bytes").item())
                n_flows_total=stats["n_flows"]
                if enable_longest_foreground_traffic:
                    data_list.append((spec,(0,n_hosts-1),topo_type_cur))
print(len(data_list))

120


In [ ]:

# [FINAL]: ns3 vs flowSim
fig_index=0
plt.figure(fig_index,figsize=(12, 6))
for spec_idx,(spec,src_dst_pair_target,topo_type) in enumerate(data_list):
    print(f"spec={spec},{topo_type}")
    dir_input_tmp = f"{dir_input}/{spec}"
    
    tmp = spec.split("_")
    n_hosts=int(tmp[2][6:])
    BDP=10*MTU
    dctcp_k_str=topo_type[9:]
    
    param_vec = np.load(f"{dir_input_tmp}/param{topo_type}.npy")
    print(f"param_vec={param_vec}")
    
    fid=np.load(f"{dir_input_tmp}/fid{topo_type}.npy")
    sizes = np.load(f"{dir_input_tmp}/fsize.npy")
    fats = np.load(f"{dir_input_tmp}/fat.npy")
    flow_src_dst=np.load(f'{dir_input_tmp}/fsd.npy')
    
    sizes=sizes[fid]
    fats=fats[fid]
    flow_src_dst=flow_src_dst[fid]
    
    flow_idx_target=np.logical_and(flow_src_dst[:,0]==src_dst_pair_target[0], flow_src_dst[:,1]==src_dst_pair_target[1])
    flow_idx_nontarget=~flow_idx_target
    flow_idx_nontarget_internal=np.logical_and(flow_src_dst[:,0]!=src_dst_pair_target[0], flow_src_dst[:,1]!=src_dst_pair_target[1])
    
    fcts = np.load(f'{dir_input_tmp}/fct{topo_type}.npy')
    i_fcts = np.load(f'{dir_input_tmp}/fct_i{topo_type}.npy')
    sldns = np.divide(fcts, i_fcts)
    # sldns = i_fcts
    print(f"sizes={len(sizes)}, fats={len(fats)}, fcts={len(fcts)}, i_fcts={len(i_fcts)}, sldns={len(sldns)}")
    deq_time_max=np.round(np.max(fats+fcts)/UNIT_G,3)
    print(f"{deq_time_max}s")
    
    if spec_idx==0:
        sizes_flowsim = np.load(f"{dir_input_tmp}/fsize.npy")
        flow_src_dst=np.load(f'{dir_input}/{spec}/fsd.npy')
        flow_idx_target_flowsim=np.logical_and(flow_src_dst[:,0]==src_dst_pair_target[0], flow_src_dst[:,1]==src_dst_pair_target[1])
    
        n_links_passed=abs(flow_src_dst[:,0]-flow_src_dst[:,1])+flow_idx_nontarget+flow_idx_nontarget_internal
        lr_bottleneck=lr
        
        delay_extra=LINK_TO_DELAY_DICT[n_hosts][flow_src_dst[:,0]]+LINK_TO_DELAY_DICT[n_hosts][flow_src_dst[:,1]]
        base_delay=get_base_delay_pmn(sizes,n_links_passed,lr_bottleneck,flow_idx_target_flowsim,flow_idx_nontarget_internal)+delay_extra
        i_fcts_flowsim = (sizes + np.ceil(sizes / MTU) * HEADER_SIZE) * BYTE_TO_BIT / lr_bottleneck + base_delay
        fcts_flowsim = np.load(f'{dir_input}/{spec}/fct_flowsim.npy')+ base_delay
        sldns_flowsim = np.divide(fcts_flowsim, i_fcts_flowsim)
        # sldns_flowsim = i_fcts_flowsim

        sizes_flowsim=sizes_flowsim[flow_idx_target]
        sldns_flowsim=sldns_flowsim[flow_idx_target_flowsim]
        fcts_flowsim=fcts_flowsim[flow_idx_target_flowsim]
        i_fcts_flowsim=i_fcts_flowsim[flow_idx_target_flowsim]
        
        bins_flowsim = np.digitize(sizes_flowsim, [MTU, BDP, 5 * BDP])

    # flow_idx_target=~flow_idx_target
    sizes=sizes[flow_idx_target]
    sldns=sldns[flow_idx_target]
    fcts=fcts[flow_idx_target]
    i_fcts=i_fcts[flow_idx_target]
    
    bins = np.digitize(sizes, [MTU, BDP, 5 * BDP])
    
    print("************************************************************************************************************************************************")
    print(f'{spec}')
    for i in range(len(labels)):
        gt = np.extract(bins==i, sldns)
        if spec_idx==0:
            gt_flowsim = np.extract(bins_flowsim==i, sldns_flowsim)
            sample_list_flowsim=np.arange(len(gt_flowsim))
            
        print(f"p99 percentile: {np.percentile(gt,99)}")
        # sample_list=sorted(np.random.choice(np.arange(len(gt)),np.min([len(gt),100000]),replace=False))
        sample_list=np.arange(len(gt))
        
        plt.subplot(2, 2, i + 1)
        plt.title(f"{labels[i]}-#{len(gt)}")
        if(len(gt)==0): continue
        # plt.xlim([0, 16])
        plt.plot(np.sort(gt)[sample_list], (np.arange(len(gt))/len(gt)*100)[sample_list], label=dctcp_k_str, linewidth=3)

        if spec_idx==0:
            plt.plot(np.sort(gt_flowsim)[sample_list_flowsim], (np.arange(len(gt_flowsim))/len(gt_flowsim)*100)[sample_list_flowsim], label="flowsim", linewidth=3,color='black')
            
        plt.axhline(99, color='green', linewidth=0.5)
        plt.xscale('log')
        plt.xlabel('Slowdown')
        plt.legend()
        plt.ylim([80, 100])
    plt.tight_layout()
fig_index+=1

plot_cdf(res,None,legend_list,'P99 slowdown',legend_font=15,group_size=len(res),loc=4,rotate_xaxis=False)